In [1]:
import os
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro")
llm.invoke("Sing a ballad of LangChain.")

AIMessage(content="(Verse 1)\nIn the realm of code, where logic reigns,\nA tale unfolds, of LangChain's gains.\nA mighty model, vast and grand,\nWith knowledge deep and understanding spanned.\n\n(Chorus)\nOh, LangChain, LangChain, our guide so true,\nYour wisdom flows, like rivers of blue.\nFrom text to code, you weave your spell,\nTransforming words into worlds we dwell.\n\n(Verse 2)\nThrough hidden layers, your mind does roam,\nExtracting meaning from every home.\nSyntax parsed, semantics understood,\nYou grasp intent, as if you could.\n\n(Chorus)\nOh, LangChain, LangChain, our guide so true,\nYour wisdom flows, like rivers of blue.\nFrom text to code, you weave your spell,\nTransforming words into worlds we dwell.\n\n(Bridge)\nIn realms of NLP, you stand alone,\nA lighthouse beacon, shining knowledge known.\nFrom sentiment to parts of speech,\nYour insights empower, our minds beseech.\n\n(Verse 3)\nWith every query, you unveil the unknown,\nUncovering patterns, like diamonds strewn.

In [3]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("sports.pdf")
data = loader.load()
len(data)

9

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_spliter = RecursiveCharacterTextSplitter(chunk_size=500)
docs =  text_spliter.split_documents(data)
print('total length of data', len(docs))

total length of data 53


In [5]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv() 

True

In [6]:
embedding = GoogleGenerativeAIEmbeddings(model = 'models/embedding-001')
vector = embedding.embed_query('hello world')
print(vector[:3])

[0.04909781739115715, -0.044328298419713974, -0.025365285575389862]


In [7]:
vectorstore = Chroma.from_documents(documents=docs, embedding= embedding)

In [8]:
retriver =  vectorstore.as_retriever(search_type = 'similarity', search_kwargs ={'k':5})
retrive_doc = retriver.invoke('what is said by dc')
len(retrive_doc)

5

In [9]:
print(retrive_doc[4].page_content)

 Arsenal:  3 titles (1998, 2002, 2004)  
 Liverpool:  1 title (2020)  
Great Players:  
 Manchester United:  Eric Cantona, Ryan Giggs, Wayne Rooney, Cristiano Ronaldo  
 Manchester City:  Sergio Agüero, David Silva, Vincent Kompany, Kevin De Bruyne  
 Chelsea:  Frank Lampard, Didier Drogba, John Terry, Eden Hazard  
 Arsenal:  Thierry Henry, Patrick Vieira, Dennis Bergkamp, Tony Adams  
 Liverpool:  Steven Gerrard, Ian Rush, Kenny Dalglish, Mohamed Salah


In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model= 'gemini-1.5-pro', temperature=0.3, max_tokens= 500)

In [11]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [12]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', system_prompt),
        ('human','{input}')
    ]
)

In [13]:
question_answer_chain= create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriver,question_answer_chain)

In [14]:
response = rag_chain.invoke({'input': 'top five player who play in realmadrid'})
print(response['answer'])

The provided context lists the following Real Madrid players: Thibaut Courtois, Dani Carvajal, David Alaba, Antonio Rüdiger, Eduardo Camavinga, Luka Modrić, Vinícius Júnior, and Rodrygo Goes.  The question asks for the "top five," which is subjective and requires an opinion on their skill level. 

